# Training and Evaluation

Still work in progress. Will be constructing a graph neural network to input the data and find the L2 orbit loss. Afterward I will also try a GAN to do the same task and compare their performances.

In [ ]:
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from matplotlib import pyplot as plt
from IPython import display
from tensorflow.examples.tutorials.mnist import input_data
import pandas as pd
import h5py
import utils
from skimage.util.shape import view_as_windows as viewW

In [17]:
def strided_indexing_roll(a, r):
    # https://stackoverflow.com/questions/20360675/roll-rows-of-a-matrix-independently
    # Concatenate with sliced to cover all rolls
    a_ext = np.concatenate((a,a[:,:-1]),axis=1)

    # Get sliding windows; use advanced-indexing to select appropriate ones
    n = a.shape[1]
    return viewW(a_ext,(1,n))[np.arange(len(r)), (n-r)%n,0]

def dense(x, scope, num_h):
    """
    standard affine layer
    scope = name tf variable scope
    num_h = number of hidden units
    """
    with tf.variable_scope(scope):
        n_x = x.get_shape().as_list()[-1]
        w = tf.get_variable('w', [n_x, num_h], initializer=tf.random_normal_initializer(stddev=0.04))
        b = tf.get_variable('b', [num_h], initializer=tf.constant_initializer(0))
        return tf.matmul(x, w)+b

def noise(X, std=1):
    """
    adds gaussian noise with unit variance
    """
    noise = tf.random_normal(shape=tf.shape(X), mean=0.0, stddev=std, dtype=tf.float32) 
    return(input_layer + noise)
    
def conv_1Ds(x, scope, filter_width, n_kernel, stride=1):
    """
    1d convolutions over multiple 1d images
    scope        = name tf variable scope
    filter_width = receptive field of kernel
    n_kernel     = # of kernels
    stride       = locations
    """
    with tf.variable_scope(scope):
        n_x = x.get_shape().as_list()[-1]
        w = tf.get_variable('w',
                            [1, filter_width, n_x, n_kernel],
                            initializer=tf.random_normal_initializer(stddev=0.04))
        b = tf.get_variable('b', [n_kernel], initializer=tf.constant_initializer(0))
        return tf.nn.convolution(x, w, padding='SAME', strides=[1, 1, stride,1])+b    
    
    
def convOneDImages(X, reuse=False):
    """
    Makes convolutions over 1D images and takes their averages
    for given object. Currently trying an approach similar to GNN.
    """
    with tf.variable_scope('convOneDImages', reuse=reuse):
        h = tf.nn.leaky_relu(conv_1Ds(X, 'c0', r_f=5, n_f=16, stride=2), 0.1) #32x16
        h = tf.nn.leaky_relu(conv_1Ds(h, 'c1', r_f=5, n_f=32, stride=2), 0.1) #16x32
        h = tf.nn.leaky_relu(conv_1Ds(h, 'c2', r_f=5, n_f=64, stride=2), 0.1) #8x64
        h = tf.nn.leaky_relu(conv_1Ds(h, 'c3', r_f=5, n_f=128, stride=2), 0.1)#4x128
        h = tf.reshape(h, [n_batch* model_n_img_per_obj, -1])
        
        h = tf.nn.leaky_relu(dense(h, 'fc_0', 128*4), 0.1)
        h = tf.nn.leaky_relu(dense(h, 'fc_1', 128*4), 0.1)
        h = tf.reshape(h, [n_batch, model_n_img_per_obj, -1]) 

        h = tf.reduce_mean(h, 1)
        return h



class objGenNetwork(object):
    """
    not finished.
    """
    def __init__(self,
                 img_dim = 64,
                 file_n_img_per_obj = 200,
                 model_n_img_per_obj = 32,
                 noise = 0.5,
                 shift_n = 9
                 minibatch_size = 50,
                 test_samples_per_batch = 10,
                 lr = 0.001):
        
        self.img_dim = img_dim
        self.file_n_img_per_obj = file_n_img_per_obj
        self.model_n_img_per_obj = model_n_img_per_obj
        self.noise = noise
        self.shift_n = shift_n
        self.minibatch_size = minibatch_size
        self.test_samples_per_batch = test_samples_per_batch
        self.lr = lr
        
        self.store = pd.HDFStore('generated_data.h5')
        self.nrows = store.get_storer('gen_2d_objects').nrows
        self.chunksize = minibatch_size + test_samples_per_batch
        
        self.gstep = tf.Variable(0, 
                                 dtype=tf.int32, 
                                 trainable=False,
                                 name='global_step')

        
    def inference(X):
        h = noise(X,self.noise)
        h = convOneDImages(X)
        h = tf.nn.leaky_relu(dense(h, 'fc_0', 1024), 0.1)
        h = tf.nn.leaky_relu(dense(h, 'fc_1', 2048), 0.1)
        self.logits = dense(h, 'p_logits', img_dim*img_dim)
        
    def data_reader(store_obj=self.store,
                    nrows = self.nrows,
                    chunksize = self.chunksize,
                    minibatch_size = self.minibatch_size,
                    test_samples_per_batch = self.test_samples_per_batch,
                    img_dim = self.img_dim,
                    noise = self.noise,
                    shift_n = self.shift_n
                    file_n_img_per_obj=self.file_n_img_per_obj,
                    model_n_img_per_obj=self.model_n_img_per_obj,
                    i=self.data_iter):
        '''
        Reads chunks of data starting for the i'th iteration.
        Prepares train_x, test_x, train_y, test_y 
        '''
        #reads the 2d objects and separates them for training and test sets.
        chunk_2d = store_obj.select('gen_2d_objects',
                                    start=i*chunksize*img_dim,
                                    stop=(i+1)*chunksize*img_dim)
        train_y=chunk_2d.iloc[:minibatch_size*img_dim,:img_dim].values.reshape([minibatch_size,\
                                                                                img_dim,img_dim]).astype('float32')
        test_y=chunk_2d.iloc[minibatch_size*img_dim:,:img_dim].values.reshape([test_samples_per_batch,\
                                                                               img_dim,img_dim]).astype('float32')
        #reads the 1d images, formats them and separates them for training and test sets.
        chunk_1d = store_obj.select('gen_1d_data',
                                    start=i*chunksize*file_n_img_per_obj,
                                    stop=(i+1)*chunksize*file_n_img_per_obj)
        # random image translations
        if(shift_n > 0):
            temp_1d_np = np.zeros((chunk_1d.shape[0],chunk_1d.shape[1]+shift_n*2))
            col_start = np.random.randint(-shift_n, shift_n, (chunk_1d.shape[0],))
            temp_1d_np[:,shift_n:img_dim+shift_n] = chunk_1d
            chunk_1d = strided_indexing_roll(temp_1d_np, col_start)[:,shift_n:img_dim+shift_n]

        # add gaussian noise
        if(noise > 0):
            chunk_1d = chunk_1d + np.random.normal(0,1,chunk_1d.shape)

            
        train_x = chunk_1d.iloc[:minibatch_size*file_n_img_per_obj,:img_dim].values.reshape([minibatch_size,\
                                                                                             file_n_img_per_obj,\
        
                                                                                             img_dim])
        test_x = chunk_1d.iloc[minibatch_size*file_n_img_per_obj:,:img_dim].values.reshape([test_samples_per_batch,\
                                                                                            file_n_img_per_obj,\
                                                                                            img_dim])
        i += 1
        #return(train_x,test_x,train_y,test_y)
        self.train_x = train_x
        self.test_x = test_x
        self.train_y = train_y
        self.test_y = test_y
        self.train_labels = train_y.reshape((minibatch_size, img_dim*img_dim))
        self.test_labels = test_y.reshape((minibatch_size, img_dim*img_dim))

    def loss(self):
        '''
        define loss function
        use softmax cross entropy with logits as the loss function
        compute mean cross entropy, softmax is applied internally
        '''
        # 
        with tf.name_scope('loss'):
            entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=self.train_labels, logits=self.logits)
            self.loss = tf.reduce_mean(entropy, name='loss')


def eval(self):
        """
        Calculates means of correct predictions per image
        in a given batch.
        """
        with tf.name_scope('predict'):           
            preds = tf.nn.sigmoid(self.logits)
            correct_preds = tf.equal(tf.round(preds), Y)
            self.accuracy = tf.reduce_mean(tf.cast(correct_preds, tf.float32))